In [1]:
import numpy as np

In [2]:
import pandas as pd


In [3]:
df = pd.read_csv('parserEcho8.csv', engine='python', encoding = 'utf-8')

In [4]:
df.head()

,Российская прыгунья в высоту Мария Ласицкене победила на чемпионате Европы по легкой атлетике в помещении.,"На турнире в Глазго, выступающая в нейтральном статусе, 26-летняя спортсменка показала результат 2,01 метра."
0,Дамаск обвинил международную коалицию во главе...,Как сообщает сирийское государственное телевид...
1,Досрочные выборы мэра польского Гданьска выигр...,б этом свидетельствуют опросы на выходе с изби...
2,Полиция США задержала в Чикаго известного певц...,Его обвиняют в десяти эпизодах сексуального на...
3,"Пассажирский самолет, по некоторым данным, вык...",Лайнер «Узбекских авиалиний» прилетел из Ташкента
4,Налоговый маневр принес российским нефтяникам ...,"Специалисты объясняют это тем, что компании по..."


In [5]:
df.shape

(108415, 2)

In [7]:
df.columns = ['title', 'description']

In [11]:
df = df.dropna(axis = 0)

In [12]:
(df['description'] == '').sum()

0

In [13]:
df['description'] = df['description'].map(lambda s: s.strip())

In [14]:
df['description'].replace('', np.nan, inplace=True)

In [15]:
df = df.dropna()

In [16]:
#df.dropna(axis=0).shape

In [17]:
df.shape

(108414, 2)

In [18]:
import nltk

In [19]:
from collections import defaultdict

In [20]:
import re

In [21]:
def clean(word):
    return re.sub('[^а-я]', '',word.lower().strip())

def tokenize(text):
    d = defaultdict(int)
    for word in text.split():
        d[clean(word)] += 1
    return d
token_des, token_title = df['description'].map(tokenize), df['title'].map(tokenize) 

In [22]:
vocab = defaultdict(int)
for d in token_des:
    for word, count in d.items():
        vocab[word] += count
for d in token_title:
    for word, count in d.items():
        vocab[word] += count

In [23]:
token_des[1]

defaultdict(int,
            {'б': 1,
             'этом': 1,
             'свидетельствуют': 1,
             'опросы': 1,
             'на': 1,
             'выходе': 1,
             'с': 1,
             'избирательных': 1,
             'участков': 1,
             'ранее': 1,
             'дулкевич': 1,
             'работала': 1,
             'заместителем': 1,
             'прежнего': 1,
             'мэра': 1,
             'павла': 1,
             'адамовича': 1,
             'который': 1,
             'был': 1,
             'убит': 1,
             'в': 1,
             'середине': 1,
             'января': 1})

In [24]:
len(vocab)

100225

In [25]:
vocab_t = (word for word, count in vocab.items() if count > 15 and len(word) > 0)
vocab_t = {word:index for index, word in enumerate(vocab_t)}
len(vocab_t)

16034

In [26]:
unk_key = vocab_t['<UNK>'] = len(vocab_t)

In [27]:
inverse_vocab = {index:word for word, index in vocab_t.items()}

In [28]:
def transform(bag):
    d = defaultdict(int)
    for word, count in bag.items():
        d[vocab_t.get(word, unk_key)] += count
    return d

In [29]:
real_des = list(map(transform, token_des))
real_title = list(map(transform, token_title))

In [30]:
real_des[0]

defaultdict(int,
            {0: 1,
             1: 1,
             2: 1,
             3: 1,
             4: 1,
             5: 1,
             6: 1,
             7: 2,
             8: 1,
             9: 1,
             16034: 1,
             10: 1,
             11: 1,
             12: 1})

In [31]:
def to_numpy(real):
    data = np.zeros((len(real), 2), dtype='int')
    for i, v in enumerate(real.items()):
        data[i] = v
    return data
nu_des = list(map(to_numpy, real_des))
nu_title = list(map(to_numpy, real_title))

In [32]:
nu_des[0]

array([[    0,     1],
       [    1,     1],
       [    2,     1],
       [    3,     1],
       [    4,     1],
       [    5,     1],
       [    6,     1],
       [    7,     2],
       [    8,     1],
       [    9,     1],
       [16034,     1],
       [   10,     1],
       [   11,     1],
       [   12,     1]])

In [33]:
eps = 10**-7

In [34]:
EMB_SIZE = 50

In [35]:
VOCAB_SIZE = len(vocab_t)

In [70]:
#embedding = np.random.randn(EMB_SIZE, VOCAB_SIZE) * 0.001
embedding = np.random.rand(EMB_SIZE, VOCAB_SIZE) * 0.001

In [71]:
matrix_delta = np.full((EMB_SIZE, VOCAB_SIZE), eps)

In [38]:
embedding.shape

(50, 16035)

In [39]:
aggreg_fun = np.mean

In [40]:
def get_repr(nu_data):
    return aggreg_fun(embedding[:, nu_data[:, 0]] * nu_data[:, 1], axis=1)

In [41]:
def get_repr_max(nu_data):
    s = embedding[:, nu_data[:, 0]] * nu_data[:, 1]
    inds = np.argmax(s, axis = 1)
    trepr = s[np.arange(EMB_SIZE), inds]
    #print("repr", trepr, nu_data[inds, 0])
    return trepr, nu_data[inds, 0]

In [42]:
get_repr(nu_des[0]).shape

(50,)

In [43]:
nu_title[0].shape

(14, 2)

In [44]:
oo = np.array(nu_title)
lens = np.array([o.shape[0] for o in oo])

In [45]:
oo2 = np.array(nu_des)
lens2 = np.array([o.shape[0] for o in oo2])

In [46]:
(lens2 == 0).sum()

0

In [47]:
oo[3884]

array([[ 1175,     1],
       [ 7923,     1],
       [11551,     1],
       [ 3901,     1],
       [   17,     1],
       [ 3120,     1],
       [    7,     1],
       [ 5320,     1],
       [  630,     1],
       [ 4002,     1],
       [  196,     1],
       [ 2633,     1],
       [ 4944,     1]])

In [48]:
token_title[3884]

defaultdict(int,
            {'премьерминистр': 1,
             'польши': 1,
             'отменил': 1,
             'поездку': 1,
             'на': 1,
             'саммит': 1,
             'в': 1,
             'иерусалим': 1,
             'после': 1,
             'высказываний': 1,
             'своего': 1,
             'израильского': 1,
             'коллеги': 1})

In [49]:
df.iloc[3884]['url']

KeyError: 'url'

In [50]:
def backward_hinge(u, v, v_hat, gamma = 1.0):
    loss = gamma - u.dot(v) + u.dot(v_hat)
    if loss > 0:
        return (v_hat - v, -u, u)

In [51]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def backward_bpr(u, v, v_hat, gamma = 1.0):
    temp = sigmoid(u.dot(v_hat) - u.dot(v))
    #u /= np.linalg.norm(u)
    #v /= np.linalg.norm(v)
    #v_hat /= np.linalg.norm(v_hat)
    #temp = sigmoid(u.dot(v) - u.dot(v_hat))
    #loss = gamma - temp
    #print(u, v, v_hat)
    #print("loss ",  - np.log(temp), u.dot(v)-u.dot(v_hat))
    #temp2 = -temp * (temp - 1)
    #temp2 = (1 - temp)
    temp2 = (temp - 1)
    temp2 = temp
    #if loss > 0:
    return (temp2 * (v_hat - v), -temp2 * u, temp2*u)
    # return (v_hat - v, -u, u)
    
def backward_bpr_2(u, v, v_hat, gamma = 1.0):
    temp = sigmoid(u.dot(v_hat) - u.dot(v))
    temp2 = (temp - 1)
    temp2 = temp
    return (temp2 * (v_hat - v), -temp2 * u, temp2*u)
    # return (v_hat - v, -u, u)

In [52]:
#adagrad
def update_old(emb, delta, idx, lam, nu):
    #p = emb[:, idx]
    #g = delta[:, idx]
    lam = lam[:, np.newaxis] 
    #print("update", p, g, lam)
    #print("update_shape", p.shape, g.shape, lam.shape, idx.shape, delta.shape, emb.shape)
    #emb[:, idx] += lam**2
    delta[:, idx] += (lam**2)
    emb[:, idx] -= (lam * nu / (delta[:, idx]**0.5 + eps))
    #delta[:, idx] +=
    #delta[:, idx] += delta

#rmsprop
def update(emb, delta, idx, lam, nu, forget=0.5):
    #p = emb[:, idx]
    #g = delta[:, idx]
    #print(lam)
    lam = lam[:, np.newaxis] 
    #print("update", p, g, lam)
    #print("update_shape", p.shape, g.shape, lam.shape, idx.shape, delta.shape, emb.shape)
    #emb[:, idx] += lam**2
    p = delta[:, idx] = forget * delta[:, idx] + (1 - forget) * (lam**2)
    emb[:, idx] -= (lam * nu / (p**0.5 + eps))
    #delta[:, idx] +=
    #delta[:, idx] += delta

#rmsprop    
def update_max(emb, delta, idx, lam, nu, forget=0.5):
    #p = emb[:, idx]
    #g = delta[:, idx]
    #print(lam)
    #lam = lam[:, np.newaxis] 
    #print("update", p, g, lam)
    #print("update_shape", p.shape, g.shape, lam.shape, idx.shape, delta.shape, emb.shape)
    #emb[:, idx] += lam**2
    ind_ar = np.arange(EMB_SIZE)
    #print(delta[ind_ar, idx].shape)
    p = delta[ind_ar, idx] = forget * delta[ind_ar, idx] + (1 - forget) * (lam**2)
    emb[ind_ar, idx] -= (lam * nu / (p**0.5 + eps))
    #delta[:, idx] +=
    #delta[:, idx] += delta

In [53]:
def magic_diff(a, b):
    r = np.setdiff1d(a[:, 0], b[:, 0], True)
    res = np.ones((r.shape[0], 2), dtype='int')
    res[:, 0] = r
    return res

In [54]:
def train(emb, delta, iu, iv_hat, nu, use_max=False):
    #print("is", iu, iv_hat)
    ind = [nu_title[iu], nu_des[iu], nu_des[iv_hat]]
    #ind[1] = ind[1].copy()
    #ind[1][:, ]
    # use real sets?
   # print("ind before", ind)
    if not True:
        if np.random.rand() < 0.5:
            ind[0] = magic_diff(ind[0], ind[1])
        else:
            ind[1] = magic_diff(ind[1], ind[0])
    if True:
        ind[1] = magic_diff(ind[1], ind[0])
        if len(ind[1]) == 0:
            return
        #ind[0] = magic_diff(ind[0], ind[1])
    #print("ind after", ind)
    infos = [None]*3
    # max thingy
    if use_max:
        vecs, infos = zip(*map(get_repr_max, ind))
        u, v, v_hat = vecs #map(get_repr, ind)
    else:
        u, v, v_hat = map(get_repr, ind)
    #print("ind repr", u, v, v_hat)
    
    hinge = backward_hinge(u, v, v_hat)
    #hinge = backward_bpr(u, v, v_hat)
    
    if hinge:
        #print('ok', ind, hinge)
        for i, lam, info in zip(ind, hinge, infos):
            #word_ind = i[:, 0]
            
            #print(lam)
            #update(emb, delta, word_ind, lam, nu)
            if use_max:
                word_ind = info
                update_max(emb, delta, word_ind, lam, nu)
            else:
                word_ind = i[:, 0]
                update(emb, delta, word_ind, lam, nu)

In [55]:
DATA_SIZE = len(nu_title)

In [56]:
TRAIN_SIZE = int(0.7 * DATA_SIZE)
TRAIN_SIZE

75889

In [57]:
TEST_SIZE = DATA_SIZE - TRAIN_SIZE

In [58]:
TEST_SIZE = 1000
TRAIN_SIZE = DATA_SIZE - TEST_SIZE

In [59]:
data_ind = np.random.permutation(DATA_SIZE)
train_ind, test_ind = data_ind[:TRAIN_SIZE], data_ind[TRAIN_SIZE:]

In [60]:
#8 => (loss = :bpr, opt = :rmsprop, agg = :max, constr = :nneg)

In [61]:
def recall_k(emb, delta, data, k=10):
    n = len(data)
    
    des_emb = np.zeros((n, EMB_SIZE))
    for i, ind in enumerate(data):
        #if i % 1000 == 0:
         #   print(i)
        des_emb[i] = get_repr(nu_des[ind])
    
    recall = 0
    
    for i, ind in enumerate(data):
        #if i % 1000 == 0:
        #    print(i)
        title_emb = get_repr(nu_title[ind])
        
        recall += (ind in data[np.argpartition(-des_emb.dot(title_emb), k)[:k]])
    return recall / n

In [62]:
def recall_k_mine(emb, delta, data, k=10, use_max = False):
    n = len(data)
    
    if use_max:
        fun_repr = lambda x: get_repr_max(x)[0]
    else:
        fun_repr = get_repr
    
    des_emb = np.zeros((n, EMB_SIZE))
    for i, ind in enumerate(data):
        #if i % 1000 == 0:
         #   print(i)
        des_emb[i] = fun_repr(nu_des[ind])
        #des_emb[i] /= np.linalg.norm(des_emb[i])
    recall = 0
    des_emb /= np.linalg.norm(des_emb, axis =0)
    
    for i, ind in enumerate(data):
        #if i % 1000 == 0:
        #    print(i)
        title_emb = fun_repr(nu_title[ind])
        title_emb /= np.linalg.norm(title_emb)
        recall += (ind in data[np.argpartition(-des_emb.dot(title_emb), k)[:k]])
    return recall / n

In [63]:
emb_copy = None

In [64]:
def epoch(emb, delta, data, nu, size=1000, use_max=False):
    rand_index_one = np.random.randint(0, TRAIN_SIZE, size=size)
    rand_index_two = np.random.randint(0, TRAIN_SIZE, size=size)
    mask = rand_index_two == rand_index_one
    rand_index_two[mask] += 1
    rand_index_two[mask] %= TRAIN_SIZE
    
    us = train_ind[rand_index_one]
    vhats = train_ind[rand_index_two]
    global emb_copy
    
    
    for i, vals in enumerate(zip(us, vhats)):
        
        u, v_hat = vals
        #print("g", (emb >= 0).sum(), (emb_copy >= 0).sum())
        #np.copyto(emb_copy, emb)
        #print("gt", (emb >= 0).sum(), (emb_copy >= 0).sum())
        train(emb, delta, u, v_hat, nu, use_max=use_max)
        if True and i % 1 == 0:
            m = emb.min()
            if m < 0:
         #       pass
                #inds = np.any(emb < 0, axis=0)
                #mins = emb[:, inds].min(axis=0)
                #emb[:, inds] -= mins
                #emb /= emb.max()
                #emb[:, u] -= m
                #emb[:, v_hat] -= m
                #mask = emb < 0
                np.clip(emb, 0, None, out=emb)
                #emb[emb < 0] = 0 #emb_copy[mask]
                #np.copyto(emb_copy, emb)
                #print("gt", (emb >= 0).sum(), (emb_copy >= 0).sum())
                #print("update", (emb >= 0).sum(), (emb_copy >= 0).sum())
                
                #print("min " , m)
        

In [65]:
import time

In [66]:
def fit(emb, delta, train_data, test_data, epoch_size=2, nu=0.01, number_epoch = 2, top_k = 10, use_max=False):
    global emb_copy
    emb_copy = np.zeros_like(emb)
    #print("g", (emb >= 0).sum(), (emb_copy >= 0).sum())
    np.copyto(emb_copy, emb)
    for i in range(number_epoch):
        start = time.time()
        epoch(emb, delta, train_data, nu, epoch_size, use_max=use_max)
        delta_time = round(time.time() - start, 3)
        score = recall_k_mine(emb, delta, test_data, top_k, use_max=use_max)
        print('Epoch {} Score {} Time {}'.format(i, score, delta_time))

#  => (loss = :bpr, opt = :rmsprop, agg = :max, constr = :nneg)#

In [67]:
#epoch(embedding, matrix_delta, train_ind, nu=0.01)

In [68]:
DATA_SIZE * 2

216828

In [69]:
fit(embedding, matrix_delta, train_ind, test_ind, number_epoch=20, nu=0.01, epoch_size=int(DATA_SIZE*2), use_max=True)

KeyboardInterrupt: 

In [62]:
EMB_SIZE = 100

In [63]:
embedding = np.random.rand(EMB_SIZE, VOCAB_SIZE) * 0.001
matrix_delta = np.full((EMB_SIZE, VOCAB_SIZE), eps)

In [72]:
fit(embedding, matrix_delta, train_ind, test_ind, number_epoch=100, nu=0.01, epoch_size=2000, use_max=not True) #int(DATA_SIZE*2), )

Epoch 0 Score 0.01 Time 5.96
Epoch 1 Score 0.01 Time 6.022
Epoch 2 Score 0.01 Time 5.49
Epoch 3 Score 0.01 Time 5.951
Epoch 4 Score 0.01 Time 5.946
Epoch 5 Score 0.01 Time 5.848
Epoch 6 Score 0.01 Time 5.795
Epoch 7 Score 0.01 Time 5.756
Epoch 8 Score 0.01 Time 5.661
Epoch 9 Score 0.01 Time 5.676
Epoch 10 Score 0.01 Time 5.567
Epoch 11 Score 0.01 Time 5.563


KeyboardInterrupt: 

In [73]:
recall_k_mine(embedding, matrix_delta, test_ind, 10)

0.01

In [74]:
embedding.shape

(50, 16035)

In [71]:
print("update", (embedding >= 0).sum(), (emb_copy >= 0).sum())

update 1239087 1239100


In [733]:
fit(embedding, matrix_delta, train_ind[:2], test_ind, number_epoch=10, nu=0.01, epoch_size=2, use_max=True) #int(DATA_SIZE*2), )

g 619550 619550
gt 619550 619550
Epoch 0 Score 0.017
gt 619550 619550
Epoch 1 Score 0.016
gt 619550 619550
Epoch 2 Score 0.019
gt 619550 619550
Epoch 3 Score 0.021
gt 619550 619550
Epoch 4 Score 0.017
gt 619550 619550
Epoch 5 Score 0.017
gt 619550 619550
Epoch 6 Score 0.013
gt 619550 619550
Epoch 7 Score 0.014
gt 619550 619550
Epoch 8 Score 0.016
gt 619550 619550
Epoch 9 Score 0.013


In [734]:
emb_copy

array([[4.56611302e-04, 6.27191100e-04, 2.81231159e-04, ...,
        1.21804960e-04, 7.75354144e-04, 1.16807692e-04],
       [6.50142437e-04, 8.77587387e-04, 7.72680039e-04, ...,
        7.06904011e-04, 9.18988977e-04, 6.00028284e-05],
       [4.08210614e-03, 3.88594587e-04, 4.85097770e-04, ...,
        7.94014404e-04, 4.54450397e-04, 6.68985460e-04],
       ...,
       [3.38400070e-04, 7.46992581e-04, 9.05357702e-04, ...,
        3.68328257e-04, 1.03444887e-04, 5.90052377e-04],
       [7.38676021e-04, 9.38138747e-04, 4.01655227e-04, ...,
        7.88108625e-04, 1.17311829e-04, 1.09398980e-04],
       [2.84485275e-03, 9.23850868e-04, 1.97345842e-04, ...,
        5.81676650e-04, 8.68193967e-04, 4.04267776e-04]])

In [75]:
embedding

array([[0.63230276, 0.22176524, 0.        , ..., 0.00266928, 0.02572171,
        0.        ],
       [0.63053312, 0.2215875 , 0.        , ..., 0.00170621, 0.02570094,
        0.        ],
       [0.63092388, 0.2213629 , 0.        , ..., 0.0017398 , 0.0256077 ,
        0.        ],
       ...,
       [0.63041051, 0.22111157, 0.        , ..., 0.00247773, 0.02560849,
        0.        ],
       [0.63127758, 0.22117524, 0.        , ..., 0.00245936, 0.02562434,
        0.        ],
       [0.63146618, 0.22182317, 0.        , ..., 0.00173934, 0.02556523,
        0.        ]])

In [662]:
matrix_delta

array([[2.34819021e-02, 3.28367475e-02, 1.31711571e-01, ...,
        1.33698132e-01, 4.09097015e-03, 6.73105668e-02],
       [8.79607261e-02, 4.33665114e-01, 1.20841208e-02, ...,
        9.80985943e-03, 7.33516438e-03, 5.05148961e-01],
       [9.09194626e-02, 3.10359041e-02, 2.27139576e-01, ...,
        9.93098992e-03, 1.00000000e-07, 1.62959707e-01],
       ...,
       [3.95050428e-01, 3.29834553e-02, 6.41709761e-03, ...,
        1.00000000e-07, 5.89042494e-03, 3.31599799e-01],
       [6.44551385e-01, 4.11284861e-02, 1.24797310e-01, ...,
        3.82658447e-02, 1.00000000e-07, 3.60542724e-01],
       [1.79829143e-01, 5.00978065e-01, 1.22946016e-01, ...,
        1.03167621e-01, 9.89489045e-03, 2.57817339e-01]])

In [503]:
a = np.arange(9).reshape((3,3))
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [505]:
np.argmax(a, axis=1)

array([2, 2, 2], dtype=int64)

In [508]:
a[np.arange(3), [2,2,2]]

array([2, 5, 8])

In [83]:
l[0] = [55,55]

In [89]:
l, a

([[55, 55], array([2, 4])], array([55, 55,  3]))

In [85]:
t = l[0]
t = [55,55]

In [88]:
a[:2] = [55,55]

In [617]:
emb_s, delta_s = embedding.copy(), matrix_delta.copy()

In [608]:
np.linalg.norm(embedding - emb_s), np.linalg.norm(matrix_delta - delta_s)

(176.00146355073906, 3.7436087400276907)

In [ ]:
a = a.astype('float')

In [ ]:
update(a, b, None, 0, 0.2)

In [151]:
a.squeeze = 'lol'

AttributeError: 'numpy.ndarray' object attribute 'squeeze' is read-only

In [148]:
a

array([55, 55,  3])

In [ ]:
b = a.copy()

In [ ]:
a, b

In [76]:
def best_k(emb, query, k=10):
    if query in vocab_t:
        ind = vocab_t[query]
        
        norm_emb = emb / np.linalg.norm(emb, axis=0)
        word_emb = norm_emb[:, ind]
        keys = np.argpartition(-word_emb.dot(norm_emb), k+1)[:k+1]
        vals = -word_emb.dot(norm_emb[:, keys])
        val_key = np.argsort(vals)[1:]
        keys = keys[val_key]
        vals = -vals[val_key]
        words = [inverse_vocab[key] for key in keys]
        print("\n".join(word + " - " + str(val) for word, val in zip(words, vals)))
    else:
        print("No such word")

In [80]:
best_k(embedding, "госдума")

аукцион - 0.9999783767177117
иск - 0.9999772426098721
его - 0.9999771341174468
форбс - 0.9999756158590289
совбезе - 0.999975596207497
виновным - 0.9999749123676254
террористических - 0.9999745657180212
суд - 0.999974197899413
террористическим - 0.999972951835542
носова - 0.9999728419541932


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [108]:
# only ind[0] update
'''
Epoch 0 Score 0.35
Epoch 1 Score 0.365
Epoch 2 Score 0.395
Epoch 3 Score 0.405
Epoch 4 Score 0.403
Epoch 5 Score 0.407
Epoch 6 Score 0.417
Epoch 7 Score 0.426
Epoch 8 Score 0.433
Epoch 9 Score 0.438'''

'\nEpoch 0 Score 0.35\nEpoch 1 Score 0.365\nEpoch 2 Score 0.395\nEpoch 3 Score 0.405\nEpoch 4 Score 0.403\nEpoch 5 Score 0.407\nEpoch 6 Score 0.417\nEpoch 7 Score 0.426\nEpoch 8 Score 0.433\nEpoch 9 Score 0.438'